# ws 01

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten #

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
target_img_shape = (64, 64)  

train_dir = './train_set'  
val_dir = './val_set'  # 

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
train_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size=target_img_shape,
                                                 batch_size=32,
                                                 class_mode='binary')


In [ ]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
val_set = val_datagen.flow_from_directory(val_dir,
                                            target_size=target_img_shape,
                                            batch_size=32,
                                            class_mode='binary')

In [ ]:
batch = train_set.next()
print(batch[0].shape)
print(batch[1].shape)

In [ ]:
print(train_set.class_indices)
for image_batch, labels_batch in train_set:
    print(image_batch.shape)
    print(labels_batch.shape)
    
    img = image_batch[0] - image_batch[0].min()
    img /= 275.0
    
    print(img.min(), img.max())
    plt.imshow(img)

    print('class:', labels_batch[0])
    break

In [ ]:
ids, counts = np.unique(train_set.classes, return_counts=True)

print(ids)
print(counts)

In [ ]:
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels   

for i in ids:
    print('{:>6} = {}' . format(labels[i], counts[i]))

In [ ]:
label_names = [k for k in train_set.class_indices]
label_names

In [ ]:
import pandas as pd

df_train_labels = pd.DataFrame({'Label':label_names, 'Count':counts})
df_train_labels.set_index('Label', inplace=True)
df_train_labels

In [ ]:
df_train_labels.plot.bar()
plt.show()

In [ ]:
ids, counts = np.unique(val_set.classes, return_counts=True)
print(ids)
print(counts)

## Model 

In [ ]:
in_shape = (target_img_shape[0], target_img_shape[1], 3)  
in_shape

In [ ]:
base_model = VGG16(include_top=False, weights='imagenet',
                   input_shape=(64, 64, 3)) 

base_model.summary()   

In [ ]:
model = Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))  # 
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
for layer in model.layers:
    print('{:12} {}'.format(layer.name, layer.trainable)) 

In [ ]:
for layer in base_model.layers:
    print('{:13} {}'.format(layer.name, layer.trainable)) 

In [ ]:
print("Trainable..\n---")
for variable in model.trainable_variables:
    print(variable.name)

In [ ]:
base_model.trainable = False 

In [ ]:
print("Trainable..\n---")
for variable in model.trainable_variables:
    print(variable.name)

In [ ]:
model.summary()

In [ ]:
import time

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', verbose=1, patience=5) # 
mc = ModelCheckpoint('TL1 tr300.h5', monitor='val_accuracy', verbose=1, save_best_only=True) #

start = time.time()
history = model.fit(train_set, 
                     validation_data=val_set, 
                    epochs=20, verbose=1, callbacks=[es, mc])

end = time.time()

print("Time Taken: {:.2f} minutes".format((end - start)/60))

In [ ]:
plt.figure(figsize=(10, 3.5))
plt.subplot(1, 2, 1)

plt.title('Loss')
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'],'r', lw=3.2, label='Validation')
plt.legend()

plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], 'r', lw=3.2, label='Validation')

plt.legend()
plt.show()

In [ ]:
acc = model.evaluate(val_set,  verbose=1)
print('score = {:.3f}' .format(acc[1]))

## Predict

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict(img_fname):
    img = load_img(img_fname, target_size=target_img_shape) 

    plt.imshow(img)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    pred = model.predict(img)

    pred_cls = 'Dog' if pred > 0.5 else 'Cat'  # use this of
    print('Prediction:',pred_cls)

In [ ]:
predict('dog010-2.jpg')

# ws 02 Fine-tune

In [ ]:
print("Trainable..\n---")
for variable in model.trainable_variables:
    print(variable.name)   

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-8]: 
    layer.trainable = False

for variable in model.trainable_variables:
    print(variable.name)

In [ ]:
model.summary()

In [ ]:
base_model.trainable = True

for layer in base_model.layers:
    # print(layer.name[:11])
    if layer.name[:11] == 'block5_conv':
        layer.trainable = True # 
        print(layer.name, '-> True')

    else:
        layer.trainable = False
        print(layer.name, '-> F') # 

In [ ]:
print("Trainable..\n---")
for variable in model.trainable_variables:
    print(variable.name)  

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
import time
start = time.time()

history = model.fit(train_set, steps_per_epoch=len(train_set),
                     validation_data=val_set, 
                     validation_steps=len(val_set), 
                    epochs=20, verbose=1, callbacks=[es, mc])

end = time.time()

print("Time Taken: {:.2f} minutes".format((end - start)/60))

In [ ]:
acc = model.evaluate(val_set, steps=len(val_set), verbose=1)
print('score = {:.3f}' .format(acc[1]))

# ws03 ResNet

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
train_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size=target_img_shape,
                                                 batch_size=32, 
                                                 class_mode='binary') 

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
    
val_set = val_datagen.flow_from_directory(val_dir,
                                            target_size=target_img_shape,
                                            batch_size=32,
                                            class_mode='binary')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


In [ ]:
base_model = ResNet50(include_top=False, 
                    weights='imagenet',
                   input_shape=(64, 64, 3))  

model = Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))  # .
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
base_model.trainable = False #  

In [ ]:
print("Trainable..\n---")
for variable in model.trainable_variables:
    print(variable.name)  